In [ ]:
import pymongo

conn_str = 'mongodb://localhost:27017/LayItOut'
client = pymongo.MongoClient(conn_str, serverSelectionTimeoutMS=60000)

print(client.server_info())

In [ ]:
db = client.get_database('LayItOut')

In [ ]:
from datetime import datetime
import doctest

start_time = datetime(2022, 6, 4, 10, 0, 0)
exp_list = [
    'yaml-list', 'let-stmt', 'match-case', 'haskell-do',
    'python-for', 'where-assign', 'yaml-map', 'python-while-list'
]
users = ['0401', '0402', '0403', '0404', '0301', '0302', '0303', '0304']

def is_manual(uid: str, exp_id: int):
    """Check if the user of uid should do a manual experiment on exp# exp_id (0..7)
    
    >>> is_manual('0401', 0)
    True
    >>> is_manual('0402', 4)
    False
    >>> is_manual('0302', 0)
    False
    >>> is_manual('0303', 6)
    True
    """
    
    return bool((int(uid[3]) ^ exp_id) & 1)

doctest.testmod()

In [ ]:
import itertools

manual_cases = list(map(lambda y: (y[0], y[1], exp_list[y[1]]), filter(lambda x: is_manual(x[0], x[1]), itertools.product(users, range(8)))))

In [ ]:
len(manual_cases)

### Manual

- take the last submission on each grammar
- time on disambiguating = time of last submission - time of first refinement

In [ ]:
manual_results = []

for uid, case_id, case in manual_cases:
    # if uid[1] == '3' and 2 <= case_id < 4: continue # skipped cases
    metrics = list(db.metricManual.find({
        "uid": uid,
        "createdAt": {"$gte": start_time},
        "grammar": case,
    }).sort("createdAt", pymongo.ASCENDING))
    result_grammar = list(db.ebnf.find({
        "uid": uid,
        "createdAt": {"$gte": metrics[-1]['createdAt']}
    }).sort("createdAt", pymongo.ASCENDING).limit(1))[0]
    manual_results.append((uid, case, result_grammar['grammarName'], result_grammar['createdAt'] - metrics[0]['createdAt']))

In [ ]:
manual_results[0], len(manual_results)

In [ ]:
import pathlib

results_root = pathlib.Path("./results/2022-06-04")
results_root.mkdir(parents=True, exist_ok=True)

In [ ]:
manual_root = results_root / "manual"
manual_root.mkdir(exist_ok=True)

for uid, grammar, grammar_id, _ in manual_results:
    g = list(db.ebnf.find({"grammarName": grammar_id}).limit(1))[0]
    filename = f"{uid}_{grammar}.ebnf"
    with (manual_root / filename).open('w') as f:
        f.write(g['ebnf'])

In [ ]:
!ls "results/2022-06-04/manual"

In [ ]:
db = client.get_database('LayItOut')

In [ ]:
from datetime import datetime
import doctest

start_time = datetime(2022, 6, 4, 10, 0, 0)
exp_list = [
    'yaml-list', 'let-stmt', 'match-case', 'haskell-do',
    'python-for', 'where-assign', 'yaml-map', 'python-while-list'
]
users = ['0401', '0402', '0403', '0404', '0301', '0302', '0303', '0304']

def is_manual(uid: str, exp_id: int):
    """Check if the user of uid should do a manual experiment on exp# exp_id (0..7)
    
    >>> is_manual('0401', 0)
    True
    >>> is_manual('0402', 4)
    False
    >>> is_manual('0302', 0)
    False
    >>> is_manual('0303', 6)
    True
    """
    
    return bool((int(uid[3]) ^ exp_id) & 1)

doctest.testmod()

In [ ]:
import itertools

manual_cases = list(map(lambda y: (y[0], y[1], exp_list[y[1]]), filter(lambda x: is_manual(x[0], x[1]), itertools.product(users, range(8)))))

In [ ]:
len(manual_cases)

In [ ]:
manual_results = []

for uid, case_id, case in manual_cases:
    # if uid[1] == '3' and 2 <= case_id < 4: continue # skipped cases
    metrics = list(db.metricManual.find({
        "uid": uid,
        "createdAt": {"$gte": start_time},
        "grammar": case,
    }).sort("createdAt", pymongo.ASCENDING))
    result_grammar = list(db.ebnf.find({
        "uid": uid,
        "createdAt": {"$gte": metrics[-1]['createdAt']}
    }).sort("createdAt", pymongo.ASCENDING).limit(1))[0]
    manual_results.append((uid, case, result_grammar['grammarName'], result_grammar['createdAt'] - metrics[0]['createdAt']))

In [ ]:
manual_results[0], len(manual_results)

In [ ]:
import pathlib

results_root = pathlib.Path("./results/2022-06-04")
results_root.mkdir(parents=True, exist_ok=True)

In [ ]:
manual_root = results_root / "manual"
manual_root.mkdir(exist_ok=True)

for uid, grammar, grammar_id, _ in manual_results:
    g = list(db.ebnf.find({"grammarName": grammar_id}).limit(1))[0]
    filename = f"{uid}_{grammar}.ebnf"
    b = manual_root / filename
    with b.open('w') as f:
        f.write(g['ebnf'])

In [ ]:
!ls "results/2022-06-04/manual"

In [ ]:
import os

check_ambiguity = True

ambig = []
for uid, grammar, _, _ in manual_results:
    print(uid, grammar)
    filename = f"{uid}_{grammar}.ebnf"
    res = os.system(f'python main.py -c 20 {str(manual_root / filename)}')
    ambig.append(res > 0)
    print(res > 0)

In [ ]:
sum(ambig), len(ambig)

There're 6 out of 32 ambiguous results in manual cases.

In [ ]:
list(filter(lambda x: x[1], zip(manual_results, ambig)))

#### P2, 0402, let-stmt, B2

```
let-expr ::= ( ( ( "let"   decl |*|  ) >> )  "in"  expr  ) >> ;
decl ::= (  expr  "="  expr  ) >> ;
expr ::= "id" | ( "id"  "id"  ) >> | let-expr;
```

#### P2, 0402, haskell-do, B4

```
document ::= stmt |+| ;
do-block ::= ( "do"  stmt |+| ) >> ;
stmt ::= ( ( "main"  "="  do-block  ) >> ) | ( "putStrLn"  "id"  ) >>;
```

#### P2, 0402, where-assign, B6

```
document ::= assign-stmt |+|;
assign-stmt ::= ( ( ( "id"  "="  expr  ) >> )  (  where ? )  ) >> ;
where ::= ( "where" assign-stmt |+|  ) >> ;
expr ::= ( "id"  "+"  "id"  ) ~ ;
```

#### P3, 0403, yaml-map, B7

```
map ::= ( ( "id"  ":"  ) ~   map-body  )|>;
map-body ::= expl-or-impl-kv |+|;
expl-kv ::= ( expl-k  expl-v?)|>;
expl-k ::= ( "?"  k-or-v  ) >> ;
expl-v ::= ( ":"  k-or-v  ) >> ;
impl-kv ::= ( ( "id"  ":"  ) ~   "id"? ) ~ ;
k-or-v ::= ( ( map-body | "id"  ) >> ) ? ;
expl-or-impl-kv ::= expl-kv | impl-kv ;
```

#### P8, 0304, haskell-do, B4

```
document ::= stmt |+|;
do-block ::= "do" -> (  stmt + );
stmt ::= (  ( "main"  "="  )>>   do-block ) | ( "putStrLn"  "id"  )>>  ;
```

#### P8, 0304, python-while-list, B8

```
document ::=  stmt |+| ;
stmt ::= "pass" | while-stmt | decl ;
while-head ::= ( ( ( "while"  "True"  ) ~ )  ":"  ) ~ ;
while-body-stmt ::= ( stmt | "break"  )>>;
while-stmt ::= ( while-head  (  while-body-stmt + )  ) >> ;
decl ::= ident  "="  list ;
ident ::= ( "foo" | "bar"  | "baz" ) ~ ;
list ::= ( (  "["   ( ( ident  ","  ) ~ ) *  ident  )>> "]"  )>>;
```

#### Inspection

- 0401
    - python-for: not enforcing sameline
- 0402
    - python-while-list: decl >>, too strict
    - other 3 are ambiguous
- 0403
    - yaml-list: wrong. disallows cascading
    - yaml-map: ambiguous
- 0404
    - all: excessive constraints
- 0304
    - let-stmt: too strict; contradicting constraints    
    - where-assign: too strict; contradicting constraints
    - python-while-list: ambiguous

### Disambiguation Time

In [ ]:
from collections import defaultdict
from datetime import timedelta

manual_time_sum = defaultdict(list)

for k, v in map(lambda x: (x[1], x[3]), manual_results):
    manual_time_sum[k].append(v)

In [ ]:
from statistics import *
from datetime import timedelta
from prettytable import PrettyTable

tbl_manual = PrettyTable()
tbl_manual.field_names = ["Grammar", "Average", "Median"]
tbl_manual.align = 'l'
tbl_manual.align['avg'] = tbl_manual.align['median'] = 'c'

for k in exp_list:
    v = manual_time_sum[k]
    tbl_manual.add_row([k, timedelta(seconds=int(fmean(map(lambda x: x.seconds, v)))), timedelta(seconds=int(median(v).seconds))])

print("Time for manual disambiguation of grammars")
print(tbl_manual)

**TODO: REMOVE TIME IN SUPPLEMENTARY MATERIAL**

### Automatic

- collect disambiguation time & final results

In [ ]:
auto_cases = list(map(lambda y: (y[0], y[1], exp_list[y[1]]), filter(lambda x: not is_manual(x[0], x[1]), itertools.product(users, range(8)))))

In [ ]:
len(auto_cases)

In [ ]:
auto_results = []

for uid, case_id, case in auto_cases:
    #if uid[1] == '3' and 2 <= case_id < 4: continue # skipped cases
    #if uid == '0404' and case_id == 6: continue # failed case
    metrics = list(db.metricTool.find({
        "uid": uid,
        "createdAt": {"$gte": start_time},
        "grammar": {"$regex": r"^" + case + r"\[........\]_round_\d+$"}
    }).sort("createdAt", pymongo.ASCENDING).limit(1))
    first_metric = metrics[0]
    result_grammar = list(db.ebnf.find({
        "uid": uid,
        "grammarName": {"$regex": r"^" + case + r"\[........\]_round_\d+$"}
    }).sort("createdAt", pymongo.DESCENDING).limit(1))[0]
    auto_results.append((uid, case, result_grammar['grammarName'], result_grammar['createdAt'] - first_metric['createdAt']))

In [ ]:
auto_results[0], len(auto_results)

In [ ]:
auto_root = results_root / "auto"
auto_root.mkdir(exist_ok=True)

for uid, grammar, grammar_id, _ in auto_results:
    g = list(db.ebnf.find({"grammarName": grammar_id}).limit(1))[0]
    filename = f"{uid}_{grammar}.ebnf"
    b = auto_root / filename
    with b.open('w') as f:
        f.write(g['ebnf'])

In [ ]:
!ls "results/2022-06-04/auto/"

In [ ]:
from collections import defaultdict
from datetime import timedelta

auto_time_sum = defaultdict(list)

for k, v in map(lambda x: (x[1], x[3]), auto_results):
    auto_time_sum[k].append(v)

In [ ]:
from statistics import *
from datetime import timedelta
from prettytable import PrettyTable

tbl_auto = PrettyTable()
tbl_auto.field_names = ["Grammar", "Average", "Median"]
tbl_auto.align = 'l'
tbl_auto.align['avg'] = tbl_auto.align['median'] = 'c'

for k in exp_list:
    v = auto_time_sum[k]
    tbl_auto.add_row([k, timedelta(seconds=int(fmean(map(lambda x: x.seconds, v)))), timedelta(seconds=int(median(v).seconds))])

print("Time for auto disambiguation of grammars")
print(tbl_auto)

In [ ]:
auto_time_sum['python-while-list']

In [ ]:
sum(auto_time_sum['python-while-list'], start=timedelta(0)) / 4

#### Inspection
- 0401
    - let-stmt: too strict, excluding examples
- 0402
    - yaml-list: wrong. disallows cascading
- 0403
    - haskell-do: excessive constraints
    - let-stmt: too strict, disallows cascading
    - python-while-list: too strict, excluding examples
    - where-assign: might be too strict, differs from examples given
- 0404
    - match-case: too weak? not requiring match -> cases
    - python-for: exc. cons.
    - yaml-list: exc. cons.
- 0301
    - where-assign: might be too strict, differs from examples given
- 0303
    - python-while-list: excluding examples
    - where-assign: exc. cons.
- 0304
    - python-for: exc. cons.
    - yaml-map: wrong. disallows cascading

#### Iteration Count

In [ ]:
for uid, case_id, case in auto_cases:
    #if uid[1] == '3' and 2 <= case_id < 4: continue # skipped cases
    #if uid == '0404' and case_id == 6: continue # failed case
    grammars = list(db.ebnf.find({
        "uid": uid,
        "grammarName": {"$regex": r"^" + case + r"\[........\]_round_\d+$"}
    }).sort("createdAt", pymongo.ASCENDING))
    print(uid, case, len(grammars))

### Paper Writing

#### time (?)

- imperfect ui design?
- iteration cnt?

#### disambiguation

- 5 / 28: effectiveness
- questionaire results

**Metrics on time are not accurate**

In [ ]:
6/32

In [2]:
db = client.get_database('LayItOut')

In [3]:
from datetime import datetime
import doctest

start_time = datetime(2022, 6, 4, 10, 0, 0)
exp_list = [
    'yaml-list', 'let-stmt', 'match-case', 'haskell-do',
    'python-for', 'where-assign', 'yaml-map', 'python-while-list'
]
users = ['0401', '0402', '0403', '0404', '0301', '0302', '0303', '0304']

def is_manual(uid: str, exp_id: int):
    """Check if the user of uid should do a manual experiment on exp# exp_id (0..7)
    
    >>> is_manual('0401', 0)
    True
    >>> is_manual('0402', 4)
    False
    >>> is_manual('0302', 0)
    False
    >>> is_manual('0303', 6)
    True
    """
    
    return bool((int(uid[3]) ^ exp_id) & 1)

doctest.testmod()

TestResults(failed=0, attempted=4)

In [4]:
import itertools

manual_cases = list(map(lambda y: (y[0], y[1], exp_list[y[1]]), filter(lambda x: is_manual(x[0], x[1]), itertools.product(users, range(8)))))

In [5]:
len(manual_cases)

32

### Manual

- take the last submission on each grammar
- time on disambiguating = time of last submission - time of first refinement

In [6]:
manual_results = []

for uid, case_id, case in manual_cases:
    # if uid[1] == '3' and 2 <= case_id < 4: continue # skipped cases
    metrics = list(db.metricManual.find({
        "uid": uid,
        "createdAt": {"$gte": start_time},
        "grammar": case,
    }).sort("createdAt", pymongo.ASCENDING))
    result_grammar = list(db.ebnf.find({
        "uid": uid,
        "createdAt": {"$gte": metrics[-1]['createdAt']}
    }).sort("createdAt", pymongo.ASCENDING).limit(1))[0]
    manual_results.append((uid, case, result_grammar['grammarName'], result_grammar['createdAt'] - metrics[0]['createdAt']))

In [7]:
manual_results[0], len(manual_results)

(('0401',
  'yaml-list',
  '__S5pW-DgBHZ7MiLjsamN9x',
  datetime.timedelta(seconds=162)),
 32)

In [8]:
import pathlib

results_root = pathlib.Path("./results/2022-06-04")
results_root.mkdir(parents=True, exist_ok=True)

In [9]:
manual_root = results_root / "manual"
manual_root.mkdir(exist_ok=True)

for uid, grammar, grammar_id, _ in manual_results:
    g = list(db.ebnf.find({"grammarName": grammar_id}).limit(1))[0]
    filename = f"{uid}_{grammar}.ebnf"
    with (manual_root / filename).open('w') as f:
        f.write(g['ebnf'])

In [10]:
!ls "results/2022-06-04/manual"

0301_match-case.ebnf	     0401_match-case.ebnf
0301_python-for.ebnf	     0401_python-for.ebnf
0301_yaml-list.ebnf	     0401_yaml-list.ebnf
0301_yaml-map.ebnf	     0401_yaml-map.ebnf
0302_haskell-do.ebnf	     0402_haskell-do.ebnf
0302_let-stmt.ebnf	     0402_let-stmt.ebnf
0302_python-while-list.ebnf  0402_python-while-list.ebnf
0302_where-assign.ebnf	     0402_where-assign.ebnf
0303_match-case.ebnf	     0403_match-case.ebnf
0303_python-for.ebnf	     0403_python-for.ebnf
0303_yaml-list.ebnf	     0403_yaml-list.ebnf
0303_yaml-map.ebnf	     0403_yaml-map.ebnf
0304_haskell-do.ebnf	     0404_haskell-do.ebnf
0304_let-stmt.ebnf	     0404_let-stmt.ebnf
0304_python-while-list.ebnf  0404_python-while-list.ebnf
0304_where-assign.ebnf	     0404_where-assign.ebnf


In [11]:
db = client.get_database('LayItOut')

In [12]:
from datetime import datetime
import doctest

start_time = datetime(2022, 6, 4, 10, 0, 0)
exp_list = [
    'yaml-list', 'let-stmt', 'match-case', 'haskell-do',
    'python-for', 'where-assign', 'yaml-map', 'python-while-list'
]
users = ['0401', '0402', '0403', '0404', '0301', '0302', '0303', '0304']

def is_manual(uid: str, exp_id: int):
    """Check if the user of uid should do a manual experiment on exp# exp_id (0..7)
    
    >>> is_manual('0401', 0)
    True
    >>> is_manual('0402', 4)
    False
    >>> is_manual('0302', 0)
    False
    >>> is_manual('0303', 6)
    True
    """
    
    return bool((int(uid[3]) ^ exp_id) & 1)

doctest.testmod()

TestResults(failed=0, attempted=4)

In [13]:
import itertools

manual_cases = list(map(lambda y: (y[0], y[1], exp_list[y[1]]), filter(lambda x: is_manual(x[0], x[1]), itertools.product(users, range(8)))))

In [14]:
len(manual_cases)

32

In [40]:
manual_results = []

for uid, case_id, case in manual_cases:
    # if uid[1] == '3' and 2 <= case_id < 4: continue # skipped cases
    metrics = list(db.metricManual.find({
        "uid": uid,
        "createdAt": {"$gte": start_time},
        "grammar": case,
    }).sort("createdAt", pymongo.ASCENDING))
    result_grammar = list(db.ebnf.find({
        "uid": uid,
        "createdAt": {"$gte": metrics[-1]['createdAt']}
    }).sort("createdAt", pymongo.ASCENDING).limit(1))[0]
    manual_results.append((uid, case, result_grammar['grammarName'], result_grammar['createdAt'] - metrics[0]['createdAt']))

In [42]:
manual_results[0], len(manual_results)

(('0401',
  'yaml-list',
  '__S5pW-DgBHZ7MiLjsamN9x',
  datetime.timedelta(seconds=162)),
 32)

In [43]:
import pathlib

results_root = pathlib.Path("./results/2022-06-04")
results_root.mkdir(parents=True, exist_ok=True)

In [44]:
manual_root = results_root / "manual"
manual_root.mkdir(exist_ok=True)

for uid, grammar, grammar_id, _ in manual_results:
    g = list(db.ebnf.find({"grammarName": grammar_id}).limit(1))[0]
    filename = f"{uid}_{grammar}.ebnf"
    b = manual_root / filename
    with b.open('w') as f:
        f.write(g['ebnf'])

In [45]:
!ls "results/2022-06-04/manual"

0301_match-case.ebnf	     0401_match-case.ebnf
0301_python-for.ebnf	     0401_python-for.ebnf
0301_yaml-list.ebnf	     0401_yaml-list.ebnf
0301_yaml-map.ebnf	     0401_yaml-map.ebnf
0302_haskell-do.ebnf	     0402_haskell-do.ebnf
0302_let-stmt.ebnf	     0402_let-stmt.ebnf
0302_python-while-list.ebnf  0402_python-while-list.ebnf
0302_where-assign.ebnf	     0402_where-assign.ebnf
0303_match-case.ebnf	     0403_match-case.ebnf
0303_python-for.ebnf	     0403_python-for.ebnf
0303_yaml-list.ebnf	     0403_yaml-list.ebnf
0303_yaml-map.ebnf	     0403_yaml-map.ebnf
0304_haskell-do.ebnf	     0404_haskell-do.ebnf
0304_let-stmt.ebnf	     0404_let-stmt.ebnf
0304_python-while-list.ebnf  0404_python-while-list.ebnf
0304_where-assign.ebnf	     0404_where-assign.ebnf


In [46]:
import os

check_ambiguity = True

ambig = []
for uid, grammar, _, _ in manual_results:
    print(uid, grammar)
    filename = f"{uid}_{grammar}.ebnf"
    res = os.system(f'python main.py -c 20 {str(manual_root / filename)}')
    ambig.append(res > 0)
    print(res > 0)

0401 yaml-list
Now checking witness sentences of length 1...
SMT variable count is 33.
extend_triple_array_by_one took 0.001592 second(s).
extend_increasing_condition_by_one took 0.005529 second(s).
extend_symbol_bound_by_one took 0.000281 second(s).
AST of the assertion deductive contains 58 nodes.
extend_deductive_condition_by_one took 0.003016 second(s).
AST of the assertion ambiguity contains 76 nodes.
build_ambiguity_condition took 0.002344 second(s).
solve took 0.000212 second(s).
Now checking witness sentences of length 2...
SMT variable count is 86.
extend_triple_array_by_one took 0.001647 second(s).
extend_increasing_condition_by_one took 0.000474 second(s).
extend_symbol_bound_by_one took 0.000218 second(s).
AST of the assertion deductive contains 186 nodes.
extend_deductive_condition_by_one took 0.004186 second(s).
AST of the assertion ambiguity contains 290 nodes.
build_ambiguity_condition took 0.004436 second(s).
solve took 0.000433 second(s).
Now checking witness sentence

/home/panda2134/.local/share/virtualenvs/smt-disambig--ZMk3Rpv/lib/python3.10/site-packages/pysmt/oracles.py:71: UserWarning: Instance-based walkers (<=0.6.0) walkers are deprecated. You should use new-style/class based walkers
  self.set_function(self.measure_to_fun[measure], *op.ALL_TYPES)


AST of the assertion ambiguity contains 3406 nodes.
build_ambiguity_condition took 0.052824 second(s).
solve took 0.022639 second(s).
Now checking witness sentences of length 7...
SMT variable count is 651.
extend_triple_array_by_one took 0.002496 second(s).
extend_increasing_condition_by_one took 0.000742 second(s).
extend_symbol_bound_by_one took 0.000273 second(s).
AST of the assertion deductive contains 2576 nodes.
extend_deductive_condition_by_one took 0.015256 second(s).
AST of the assertion ambiguity contains 4970 nodes.
build_ambiguity_condition took 0.038002 second(s).
solve took 0.032781 second(s).
Now checking witness sentences of length 8...
SMT variable count is 824.
extend_triple_array_by_one took 0.002804 second(s).
extend_increasing_condition_by_one took 0.000422 second(s).
extend_symbol_bound_by_one took 0.000201 second(s).
AST of the assertion deductive contains 3544 nodes.
extend_deductive_condition_by_one took 0.014639 second(s).
AST of the assertion ambiguity conta

/home/panda2134/.local/share/virtualenvs/smt-disambig--ZMk3Rpv/lib/python3.10/site-packages/pysmt/oracles.py:71: UserWarning: Instance-based walkers (<=0.6.0) walkers are deprecated. You should use new-style/class based walkers
  self.set_function(self.measure_to_fun[measure], *op.ALL_TYPES)


solve took 0.006497 second(s).
Now checking witness sentences of length 6...
SMT variable count is 930.
extend_triple_array_by_one took 0.003045 second(s).
extend_increasing_condition_by_one took 0.000453 second(s).
extend_symbol_bound_by_one took 0.000284 second(s).
AST of the assertion deductive contains 3738 nodes.
extend_deductive_condition_by_one took 0.019909 second(s).
AST of the assertion ambiguity contains 6303 nodes.
build_ambiguity_condition took 0.045778 second(s).
solve took 0.017983 second(s).
Now checking witness sentences of length 7...
SMT variable count is 1218.
extend_triple_array_by_one took 0.003286 second(s).
extend_increasing_condition_by_one took 0.000518 second(s).
extend_symbol_bound_by_one took 0.000268 second(s).
AST of the assertion deductive contains 5488 nodes.
extend_deductive_condition_by_one took 0.024224 second(s).
AST of the assertion ambiguity contains 9894 nodes.
build_ambiguity_condition took 0.067601 second(s).
solve took 0.035405 second(s).
Now 

...

solve took 0.009877 second(s).
Now checking witness sentences of length 6...
SMT variable count is 882.
extend_triple_array_by_one took 0.003069 second(s).
extend_increasing_condition_by_one took 0.000443 second(s).
extend_symbol_bound_by_one took 0.000214 second(s).
AST of the assertion deductive contains 3718 nodes.
extend_deductive_condition_by_one took 0.019760 second(s).
AST of the assertion ambiguity contains 6203 nodes.
build_ambiguity_condition took 0.045933 second(s).
solve took 0.021286 second(s).
Now checking witness sentences of length 7...
SMT variable count is 1155.
extend_triple_array_by_one took 0.002993 second(s).
extend_increasing_condition_by_one took 0.000372 second(s).
extend_symbol_bound_by_one took 0.000197 second(s).
AST of the assertion deductive contains 5460 nodes.
extend_deductive_condition_by_one took 0.022509 second(s).
AST of the assertion ambiguity contains 9574 nodes.
build_ambiguity_condition took 0.067201 second(s).
solve took 0.048144 second(s).
Foun

/home/panda2134/.local/share/virtualenvs/smt-disambig--ZMk3Rpv/lib/python3.10/site-packages/pysmt/oracles.py:71: UserWarning: Instance-based walkers (<=0.6.0) walkers are deprecated. You should use new-style/class based walkers
  self.set_function(self.measure_to_fun[measure], *op.ALL_TYPES)


AST of the assertion ambiguity contains 4358 nodes.
build_ambiguity_condition took 0.066360 second(s).
solve took 0.007601 second(s).
Now checking witness sentences of length 6...
SMT variable count is 1266.
extend_triple_array_by_one took 0.003579 second(s).
extend_increasing_condition_by_one took 0.000379 second(s).
extend_symbol_bound_by_one took 0.000176 second(s).
AST of the assertion deductive contains 5786 nodes.
extend_deductive_condition_by_one took 0.027588 second(s).
AST of the assertion ambiguity contains 7279 nodes.
build_ambiguity_condition took 0.059876 second(s).
solve took 0.022010 second(s).
Now checking witness sentences of length 7...
SMT variable count is 1659.
extend_triple_array_by_one took 0.004143 second(s).
extend_increasing_condition_by_one took 0.000485 second(s).
extend_symbol_bound_by_one took 0.000216 second(s).
AST of the assertion deductive contains 8512 nodes.
extend_deductive_condition_by_one took 0.033216 second(s).
AST of the assertion ambiguity con

/home/panda2134/.local/share/virtualenvs/smt-disambig--ZMk3Rpv/lib/python3.10/site-packages/pysmt/oracles.py:71: UserWarning: Instance-based walkers (<=0.6.0) walkers are deprecated. You should use new-style/class based walkers
  self.set_function(self.measure_to_fun[measure], *op.ALL_TYPES)


AST of the assertion ambiguity contains 4850 nodes.
build_ambiguity_condition took 0.080519 second(s).
solve took 0.018748 second(s).
Now checking witness sentences of length 5...
SMT variable count is 1555.
extend_triple_array_by_one took 0.004711 second(s).
extend_increasing_condition_by_one took 0.000533 second(s).
extend_symbol_bound_by_one took 0.000240 second(s).
AST of the assertion deductive contains 5530 nodes.
extend_deductive_condition_by_one took 0.037984 second(s).
AST of the assertion ambiguity contains 8492 nodes.
build_ambiguity_condition took 0.083596 second(s).
solve took 0.047425 second(s).
Found a witness sentence, continue to check if it's a false positive.
AST of the assertion sentence reachable contains 6189 nodes.
build_reachable_condition took 0.088144 second(s).
solve took 0.003636 second(s).
Ambiguous sentence of length 5 found.
True


In [60]:
sum(ambig), len(ambig)

(6, 32)

There're 6 out of 32 ambiguous results in manual cases.

In [80]:
list(filter(lambda x: x[1], zip(manual_results, ambig)))

[(('0402',
   'let-stmt',
   '__JA3KhnErDU8J5_rRcEGrr',
   datetime.timedelta(seconds=557)),
  True),
 (('0402',
   'haskell-do',
   '__oiONQO1rKV-enKQX2LgzM',
   datetime.timedelta(seconds=152)),
  True),
 (('0402',
   'where-assign',
   '__pgodrfWWoS3QIanDpQIj1',
   datetime.timedelta(seconds=94)),
  True),
 (('0403',
   'yaml-map',
   '__b_ajbentO29f3gos2kZNr',
   datetime.timedelta(seconds=415)),
  True),
 (('0304',
   'haskell-do',
   '__Zsdc-OdhGXMW9PKNarEcx',
   datetime.timedelta(seconds=181)),
  True),
 (('0304',
   'python-while-list',
   '__3v0AYkpOGXgqGOnY0Ie1H',
   datetime.timedelta(seconds=402)),
  True)]

#### P2, 0402, let-stmt, B2

```
let-expr ::= ( ( ( "let"   decl |*|  ) >> )  "in"  expr  ) >> ;
decl ::= (  expr  "="  expr  ) >> ;
expr ::= "id" | ( "id"  "id"  ) >> | let-expr;
```

#### P2, 0402, haskell-do, B4

```
document ::= stmt |+| ;
do-block ::= ( "do"  stmt |+| ) >> ;
stmt ::= ( ( "main"  "="  do-block  ) >> ) | ( "putStrLn"  "id"  ) >>;
```

#### P2, 0402, where-assign, B6

```
document ::= assign-stmt |+|;
assign-stmt ::= ( ( ( "id"  "="  expr  ) >> )  (  where ? )  ) >> ;
where ::= ( "where" assign-stmt |+|  ) >> ;
expr ::= ( "id"  "+"  "id"  ) ~ ;
```

#### P3, 0403, yaml-map, B7

```
map ::= ( ( "id"  ":"  ) ~   map-body  )|>;
map-body ::= expl-or-impl-kv |+|;
expl-kv ::= ( expl-k  expl-v?)|>;
expl-k ::= ( "?"  k-or-v  ) >> ;
expl-v ::= ( ":"  k-or-v  ) >> ;
impl-kv ::= ( ( "id"  ":"  ) ~   "id"? ) ~ ;
k-or-v ::= ( ( map-body | "id"  ) >> ) ? ;
expl-or-impl-kv ::= expl-kv | impl-kv ;
```

#### P8, 0304, haskell-do, B4

```
document ::= stmt |+|;
do-block ::= "do" -> (  stmt + );
stmt ::= (  ( "main"  "="  )>>   do-block ) | ( "putStrLn"  "id"  )>>  ;
```

#### P8, 0304, python-while-list, B8

```
document ::=  stmt |+| ;
stmt ::= "pass" | while-stmt | decl ;
while-head ::= ( ( ( "while"  "True"  ) ~ )  ":"  ) ~ ;
while-body-stmt ::= ( stmt | "break"  )>>;
while-stmt ::= ( while-head  (  while-body-stmt + )  ) >> ;
decl ::= ident  "="  list ;
ident ::= ( "foo" | "bar"  | "baz" ) ~ ;
list ::= ( (  "["   ( ( ident  ","  ) ~ ) *  ident  )>> "]"  )>>;
```

#### Inspection

- 0401
    - python-for: not enforcing sameline
- 0402
    - python-while-list: decl >>, too strict
    - other 3 are ambiguous
- 0403
    - yaml-list: wrong. disallows cascading
    - yaml-map: ambiguous
- 0404
    - all: excessive constraints
- 0304
    - let-stmt: too strict; contradicting constraints    
    - where-assign: too strict; contradicting constraints
    - python-while-list: ambiguous

### Disambiguation Time

In [62]:
from collections import defaultdict
from datetime import timedelta

manual_time_sum = defaultdict(list)

for k, v in map(lambda x: (x[1], x[3]), manual_results):
    manual_time_sum[k].append(v)

In [77]:
from statistics import *
from datetime import timedelta
from prettytable import PrettyTable

tbl_manual = PrettyTable()
tbl_manual.field_names = ["Grammar", "Average", "Median"]
tbl_manual.align = 'l'
tbl_manual.align['avg'] = tbl_manual.align['median'] = 'c'

for k in exp_list:
    v = manual_time_sum[k]
    tbl_manual.add_row([k, timedelta(seconds=int(fmean(map(lambda x: x.seconds, v)))), timedelta(seconds=int(median(v).seconds))])

print("Time for manual disambiguation of grammars")
print(tbl_manual)

Time for manual disambiguation of grammars
+-------------------+---------+---------+
| Grammar           | Average | Median  |
+-------------------+---------+---------+
| yaml-list         | 0:02:25 | 0:02:07 |
| let-stmt          | 0:06:45 | 0:06:56 |
| match-case        | 0:01:53 | 0:01:32 |
| haskell-do        | 0:02:28 | 0:02:46 |
| python-for        | 0:01:33 | 0:01:26 |
| where-assign      | 0:02:35 | 0:02:29 |
| yaml-map          | 0:05:11 | 0:04:55 |
| python-while-list | 0:08:20 | 0:05:53 |
+-------------------+---------+---------+


**TODO: REMOVE TIME IN SUPPLEMENTARY MATERIAL**

### Automatic

- collect disambiguation time & final results

In [64]:
auto_cases = list(map(lambda y: (y[0], y[1], exp_list[y[1]]), filter(lambda x: not is_manual(x[0], x[1]), itertools.product(users, range(8)))))

In [65]:
len(auto_cases)

32

In [66]:
auto_results = []

for uid, case_id, case in auto_cases:
    #if uid[1] == '3' and 2 <= case_id < 4: continue # skipped cases
    #if uid == '0404' and case_id == 6: continue # failed case
    metrics = list(db.metricTool.find({
        "uid": uid,
        "createdAt": {"$gte": start_time},
        "grammar": {"$regex": r"^" + case + r"\[........\]_round_\d+$"}
    }).sort("createdAt", pymongo.ASCENDING).limit(1))
    first_metric = metrics[0]
    result_grammar = list(db.ebnf.find({
        "uid": uid,
        "grammarName": {"$regex": r"^" + case + r"\[........\]_round_\d+$"}
    }).sort("createdAt", pymongo.DESCENDING).limit(1))[0]
    auto_results.append((uid, case, result_grammar['grammarName'], result_grammar['createdAt'] - first_metric['createdAt']))

In [67]:
auto_results[0], len(auto_results)

(('0401',
  'let-stmt',
  'let-stmt[4JgUWjlB]_round_2',
  datetime.timedelta(seconds=79)),
 32)

In [68]:
auto_root = results_root / "auto"
auto_root.mkdir(exist_ok=True)

for uid, grammar, grammar_id, _ in auto_results:
    g = list(db.ebnf.find({"grammarName": grammar_id}).limit(1))[0]
    filename = f"{uid}_{grammar}.ebnf"
    b = auto_root / filename
    with b.open('w') as f:
        f.write(g['ebnf'])

In [69]:
!ls "results/2022-06-04/auto/"

0301_haskell-do.ebnf	     0401_haskell-do.ebnf
0301_let-stmt.ebnf	     0401_let-stmt.ebnf
0301_python-while-list.ebnf  0401_python-while-list.ebnf
0301_where-assign.ebnf	     0401_where-assign.ebnf
0302_match-case.ebnf	     0402_match-case.ebnf
0302_python-for.ebnf	     0402_python-for.ebnf
0302_yaml-list.ebnf	     0402_yaml-list.ebnf
0302_yaml-map.ebnf	     0402_yaml-map.ebnf
0303_haskell-do.ebnf	     0403_haskell-do.ebnf
0303_let-stmt.ebnf	     0403_let-stmt.ebnf
0303_python-while-list.ebnf  0403_python-while-list.ebnf
0303_where-assign.ebnf	     0403_where-assign.ebnf
0304_match-case.ebnf	     0404_match-case.ebnf
0304_python-for.ebnf	     0404_python-for.ebnf
0304_yaml-list.ebnf	     0404_yaml-list.ebnf
0304_yaml-map.ebnf	     0404_yaml-map.ebnf


In [70]:
from collections import defaultdict
from datetime import timedelta

auto_time_sum = defaultdict(list)

for k, v in map(lambda x: (x[1], x[3]), auto_results):
    auto_time_sum[k].append(v)

In [71]:
from statistics import *
from datetime import timedelta
from prettytable import PrettyTable

tbl_auto = PrettyTable()
tbl_auto.field_names = ["Grammar", "Average", "Median"]
tbl_auto.align = 'l'
tbl_auto.align['avg'] = tbl_auto.align['median'] = 'c'

for k in exp_list:
    v = auto_time_sum[k]
    tbl_auto.add_row([k, timedelta(seconds=int(fmean(map(lambda x: x.seconds, v)))), timedelta(seconds=int(median(v).seconds))])

print("Time for auto disambiguation of grammars")
print(tbl_auto)

Time for auto disambiguation of grammars
+-------------------+---------+---------+
| Grammar           | Average | Median  |
+-------------------+---------+---------+
| yaml-list         | 0:02:57 | 0:02:47 |
| let-stmt          | 0:01:57 | 0:01:50 |
| match-case        | 0:06:09 | 0:05:22 |
| haskell-do        | 0:02:24 | 0:02:04 |
| python-for        | 0:01:56 | 0:01:51 |
| where-assign      | 0:03:54 | 0:03:42 |
| yaml-map          | 0:11:02 | 0:08:24 |
| python-while-list | 0:01:54 | 0:01:54 |
+-------------------+---------+---------+


In [72]:
auto_time_sum['python-while-list']

[datetime.timedelta(seconds=119),
 datetime.timedelta(seconds=110),
 datetime.timedelta(seconds=61),
 datetime.timedelta(seconds=167)]

In [73]:
sum(auto_time_sum['python-while-list'], start=timedelta(0)) / 4

datetime.timedelta(seconds=114, microseconds=250000)

#### Inspection
- 0401
    - let-stmt: too strict, excluding examples
- 0402
    - yaml-list: wrong. disallows cascading
- 0403
    - haskell-do: excessive constraints
    - let-stmt: too strict, disallows cascading
    - python-while-list: too strict, excluding examples
    - where-assign: might be too strict, differs from examples given
- 0404
    - match-case: too weak? not requiring match -> cases
    - python-for: exc. cons.
    - yaml-list: exc. cons.
- 0301
    - where-assign: might be too strict, differs from examples given
- 0303
    - python-while-list: excluding examples
    - where-assign: exc. cons.
- 0304
    - python-for: exc. cons.
    - yaml-map: wrong. disallows cascading

#### Iteration Count

In [74]:
for uid, case_id, case in auto_cases:
    #if uid[1] == '3' and 2 <= case_id < 4: continue # skipped cases
    #if uid == '0404' and case_id == 6: continue # failed case
    grammars = list(db.ebnf.find({
        "uid": uid,
        "grammarName": {"$regex": r"^" + case + r"\[........\]_round_\d+$"}
    }).sort("createdAt", pymongo.ASCENDING))
    print(uid, case, len(grammars))

0401 let-stmt 2
0401 haskell-do 3
0401 where-assign 3
0401 python-while-list 2
0402 yaml-list 3
0402 match-case 3
0402 python-for 2
0402 yaml-map 11
0403 let-stmt 2
0403 haskell-do 2
0403 where-assign 4
0403 python-while-list 2
0404 yaml-list 2
0404 match-case 3
0404 python-for 2
0404 yaml-map 8
0301 let-stmt 2
0301 haskell-do 2
0301 where-assign 2
0301 python-while-list 2
0302 yaml-list 2
0302 match-case 5
0302 python-for 2
0302 yaml-map 3
0303 let-stmt 2
0303 haskell-do 2
0303 where-assign 2
0303 python-while-list 2
0304 yaml-list 3
0304 match-case 2
0304 python-for 3
0304 yaml-map 5


### Paper Writing

#### time (?)

- imperfect ui design?
- iteration cnt?

#### disambiguation

- 5 / 28: effectiveness
- questionaire results

**Metrics on time are not accurate**

In [75]:
6/32

0.1875